In [ ]:
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_CONTOURS

PARAMETROS MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic   #todo el cuerpo
mp_drawing = mp.solutions.drawing_utils # DIBUJA LOS PUNTOS DEL CUERPO

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #CONVERSION COLOR BGR 2 RGB
    image.flags.writeable = False                  # Escritura de imagen anulada(False)
    results = model.process(image)                 # Hacer deteccion
    image.flags.writeable = True                   # Escritura de imagen activada(True) 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # CONVERSION COLOR RGB 2 BGR
    return image, results

DIBUJA PUNTOS DE LAS DIFERENTES PARTES

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, FACEMESH_CONTOURS) 
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 

DIBUJA CONEXIONES ENTRE PUNTOS 

In [6]:
def draw_styled_landmarks(image, results):
    # conexiones cara
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # conexiones cuerpo
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # conexiones mano izquierda
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # conexiones mano derecha  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

ACTIVAR CAMARA Y DETECCIONES

In [6]:

cap = cv2.VideoCapture(0)   # inicializar captura puerto 0

# Inicializar modelo holistico
with mp_holistic.Holistic(static_image_mode=False, model_complexity=1) as holistic:
    
    while cap.isOpened():

        # lectura de camara
        ret, frame = cap.read()

        # Hacer detecciones
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Dibujar landmarks( puntos y rayas)
        draw_styled_landmarks(image, results)

        # Ver en pantalla
        cv2.imshow('OpenCV Feed', image)

        # Para apagar imagen,  (0XFF)es codigo parq 64bits
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()   # finalizar captura
    cv2.destroyAllWindows()  # cerrar cualquier ventana abierta

In [ ]:
draw_landmarks(frame, results) # dibuja los puntos del video)

COMPROBACIONES

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) # ver captura de video

In [31]:
#results.pose_landmarks.landmark[0].visibility

In [ ]:
len(results.left_hand_landmarks.landmark) # puntos que ha detectado en la capturade la mano izquierda)
                                            

EXTRAER VALORES DE LOS PUNTOS CLAVE

In [ ]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [12]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [13]:
# face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() 
#     if results.face_landmarks 
#     else np.zeros(1404)


In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [15]:
result_test = extract_keypoints(results)

In [16]:
result_test

array([ 0.72696429,  0.83596051, -0.6263485 , ...,  0.        ,
        0.        ,  0.        ])

In [17]:
#grabacion test(directorio jupyter)
np.save('0', result_test)

In [18]:
#cargar test
np.load('0.npy')

array([ 0.72696429,  0.83596051, -0.6263485 , ...,  0.        ,
        0.        ,  0.        ])

# CONFIGURAR CARPETAS PARA SELECCION DE ELEMENTOS

In [61]:
# DONDE BUSCAR LOS DATOS
DATA_PATH = os.path.join('./reconocimiento_imagenes/DATA/') 

# ACCIONES A DETECTAR
actions = np.array(['colegio','abajo'])

# Thirty videos worth of data
no_sequences = 50

# Videos are going to be 30 frames in length
sequence_length = 50

# Folder start
start_folder =0

In [62]:
#creamos 30 carpetas de cada elemento en la carpeta asignada, luego anulo la opcion este bloque y continuo

for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [104]:
# #al haber creado las carpetas antes, ya no da error esto
# # esto tambien hay que anularlo porque crea las carpetas mal despues.
# for action in actions: 
#     dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
#     for sequence in range(1,no_sequences+1):
#         try: 
#             os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
#         except:
#             pass

# RECOPILACION DE IMAGENES PARA LUEGO TRATARLAS MAS ADELANTE

En cuanto actives la recopilacion, empezara por la primera que has marcado 
en mi caso (colegio) y se iran grabando fotogramas y seguira grabando hasta 
que llege a 30, luego seguira con la siguiente imagen (sentir), a asi sucesivamente hasta que 
se acaben las imagenes que hemos marcado.
Esto se grabara en las carpetas que antes hemos creado

In [63]:
cap = cv2.VideoCapture(0)

# Inicializar modelo holistico 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NUEVO BUCLE
    
    # RECORRER ACCIONES
    for action in actions:
        # RECORRER LAS SECUENCIAS DE VIDEO
        for sequence in range(start_folder, start_folder+no_sequences):
            # RECORRER LA DURACION DE LA SECUENCIA DE VIDEO
            for frame_num in range(sequence_length):

                #lectura de camara
                ret, frame = cap.read()

                # Hacer detecciones
                image, results = mediapipe_detection(frame, holistic)

                # Dibujar landmarks
                draw_styled_landmarks(image, results)
                
                
                # ( Aplicar logica en espera)
                if frame_num == 0: 
                    cv2.putText(image, 'COMENZAR COLECCION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Recopilando frames para {} Video Numero {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Mostrar en pantalla
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Recopilando frames para {} Video Numero {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Mostrar en pantalla
                    cv2.imshow('OpenCV Feed', image)
                
                # Exportar puntos clave
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # apagar camara
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [64]:
cap.release()
cv2.destroyAllWindows()

# PREPROCESAR DATOS, CREAR ETIQUETAS Y FUNCIONES

In [65]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [66]:
# para ver etiquetas 
label_map = {label:num for num, label in enumerate(actions)}

In [67]:
label_map

{'colegio': 0, 'abajo': 1}

In [68]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [69]:
np.array(sequences).shape

(100, 50, 1662)

In [70]:
np.array(labels).shape

(100,)

In [71]:
X = np.array(sequences)


In [72]:
X.shape

(100, 50, 1662)

In [73]:
y = to_categorical(labels).astype(int)

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [75]:
y_test.shape

(5, 2)

# CONSTRUIR Y ENTRENAR LA RED NEURONAL

In [76]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [77]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [78]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(50,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [79]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [80]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 2s 85ms/step - loss: 2.1927 - categorical_accuracy: 0.4421
Epoch 2/2000
3/3 [==============================] - 0s 76ms/step - loss: 1.3631 - categorical_accuracy: 0.5263
Epoch 3/2000
3/3 [==============================] - 0s 81ms/step - loss: 1.1839 - categorical_accuracy: 0.5263
Epoch 4/2000
3/3 [==============================] - 0s 77ms/step - loss: 0.9263 - categorical_accuracy: 0.5263
Epoch 5/2000
3/3 [==============================] - 0s 77ms/step - loss: 5.2195 - categorical_accuracy: 0.5263
Epoch 6/2000
3/3 [==============================] - 0s 77ms/step - loss: 18.6491 - categorical_accuracy: 0.5684
Epoch 7/2000
3/3 [==============================] - 0s 74ms/step - loss: 17.8314 - categorical_accuracy: 0.4737
Epoch 8/2000
3/3 [==============================] - 0s 76ms/step - loss: 20.9295 - categorical_accuracy: 0.4211
Epoch 9/2000
3/3 [==============================] - 0s 76ms/step - loss: 11.5728 - categorical_accuracy: 0.52

3/3 [==============================] - 0s 83ms/step - loss: 0.3273 - categorical_accuracy: 0.8737
Epoch 74/2000
3/3 [==============================] - 0s 78ms/step - loss: 0.3199 - categorical_accuracy: 0.8632
Epoch 75/2000
3/3 [==============================] - 0s 81ms/step - loss: 0.3167 - categorical_accuracy: 0.9158
Epoch 76/2000
3/3 [==============================] - 0s 78ms/step - loss: 0.3195 - categorical_accuracy: 0.9053
Epoch 77/2000
3/3 [==============================] - 0s 79ms/step - loss: 0.3117 - categorical_accuracy: 0.8737
Epoch 78/2000
3/3 [==============================] - 0s 78ms/step - loss: 0.3045 - categorical_accuracy: 0.9053
Epoch 79/2000
3/3 [==============================] - 0s 84ms/step - loss: 0.3106 - categorical_accuracy: 0.9053
Epoch 80/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.3043 - categorical_accuracy: 0.8842
Epoch 81/2000
3/3 [==============================] - 0s 79ms/step - loss: 0.3052 - categorical_accuracy: 0.9158
Epoch 

3/3 [==============================] - 0s 75ms/step - loss: 0.2683 - categorical_accuracy: 0.9158
Epoch 146/2000
3/3 [==============================] - 0s 81ms/step - loss: 0.2633 - categorical_accuracy: 0.8947
Epoch 147/2000
3/3 [==============================] - 0s 76ms/step - loss: 0.2385 - categorical_accuracy: 0.9053
Epoch 148/2000
3/3 [==============================] - 0s 83ms/step - loss: 0.2270 - categorical_accuracy: 0.9158
Epoch 149/2000
3/3 [==============================] - 0s 77ms/step - loss: 0.2327 - categorical_accuracy: 0.9158
Epoch 150/2000
3/3 [==============================] - 0s 77ms/step - loss: 0.2256 - categorical_accuracy: 0.9053
Epoch 151/2000
3/3 [==============================] - 0s 80ms/step - loss: 0.2340 - categorical_accuracy: 0.9158
Epoch 152/2000
3/3 [==============================] - 0s 79ms/step - loss: 0.2433 - categorical_accuracy: 0.9158
Epoch 153/2000
3/3 [==============================] - 0s 80ms/step - loss: 0.2476 - categorical_accuracy: 0.905

3/3 [==============================] - 0s 79ms/step - loss: 0.2093 - categorical_accuracy: 0.9053
Epoch 218/2000
3/3 [==============================] - 0s 80ms/step - loss: 0.2129 - categorical_accuracy: 0.9263
Epoch 219/2000
3/3 [==============================] - 0s 79ms/step - loss: 0.1996 - categorical_accuracy: 0.9053
Epoch 220/2000
3/3 [==============================] - 0s 75ms/step - loss: 0.2300 - categorical_accuracy: 0.9053
Epoch 221/2000
3/3 [==============================] - 0s 74ms/step - loss: 0.2696 - categorical_accuracy: 0.9053
Epoch 222/2000
3/3 [==============================] - 0s 78ms/step - loss: 0.2116 - categorical_accuracy: 0.9263
Epoch 223/2000
3/3 [==============================] - 0s 91ms/step - loss: 0.2317 - categorical_accuracy: 0.8947
Epoch 224/2000
3/3 [==============================] - 0s 74ms/step - loss: 0.1963 - categorical_accuracy: 0.9053
Epoch 225/2000
3/3 [==============================] - 0s 74ms/step - loss: 0.2202 - categorical_accuracy: 0.905

3/3 [==============================] - 0s 83ms/step - loss: 0.2280 - categorical_accuracy: 0.9158
Epoch 290/2000
3/3 [==============================] - 0s 90ms/step - loss: 0.1470 - categorical_accuracy: 0.9368
Epoch 291/2000
3/3 [==============================] - 0s 94ms/step - loss: 0.2121 - categorical_accuracy: 0.9474
Epoch 292/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.1771 - categorical_accuracy: 0.9158
Epoch 293/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.1642 - categorical_accuracy: 0.9474
Epoch 294/2000
3/3 [==============================] - 0s 91ms/step - loss: 0.1836 - categorical_accuracy: 0.9368
Epoch 295/2000
3/3 [==============================] - 0s 102ms/step - loss: 0.1524 - categorical_accuracy: 0.9263
Epoch 296/2000
3/3 [==============================] - 0s 95ms/step - loss: 0.1559 - categorical_accuracy: 0.9263
Epoch 297/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.1588 - categorical_accuracy: 0.92

...

3/3 [==============================] - 0s 77ms/step - loss: 3.3461e-04 - categorical_accuracy: 1.0000
Epoch 1712/2000
3/3 [==============================] - 0s 78ms/step - loss: 3.3328e-04 - categorical_accuracy: 1.0000
Epoch 1713/2000
3/3 [==============================] - 0s 78ms/step - loss: 3.3432e-04 - categorical_accuracy: 1.0000
Epoch 1714/2000
3/3 [==============================] - 0s 77ms/step - loss: 3.3400e-04 - categorical_accuracy: 1.0000
Epoch 1715/2000
3/3 [==============================] - 0s 77ms/step - loss: 3.3150e-04 - categorical_accuracy: 1.0000
Epoch 1716/2000
3/3 [==============================] - 0s 77ms/step - loss: 3.3182e-04 - categorical_accuracy: 1.0000
Epoch 1717/2000
3/3 [==============================] - 0s 79ms/step - loss: 3.3009e-04 - categorical_accuracy: 1.0000
Epoch 1718/2000
3/3 [==============================] - 0s 78ms/step - loss: 3.3157e-04 - categorical_accuracy: 1.0000
Epoch 1719/2000
3/3 [==============================] - 0s 82ms/step - lo

3/3 [==============================] - 0s 76ms/step - loss: 2.8846e-04 - categorical_accuracy: 1.0000
Epoch 1781/2000
3/3 [==============================] - 0s 79ms/step - loss: 2.8819e-04 - categorical_accuracy: 1.0000
Epoch 1782/2000
3/3 [==============================] - 0s 78ms/step - loss: 2.8480e-04 - categorical_accuracy: 1.0000
Epoch 1783/2000
3/3 [==============================] - 0s 77ms/step - loss: 2.8684e-04 - categorical_accuracy: 1.0000
Epoch 1784/2000
3/3 [==============================] - 0s 76ms/step - loss: 2.8603e-04 - categorical_accuracy: 1.0000
Epoch 1785/2000
3/3 [==============================] - 0s 76ms/step - loss: 2.8411e-04 - categorical_accuracy: 1.0000
Epoch 1786/2000
3/3 [==============================] - 0s 78ms/step - loss: 2.8419e-04 - categorical_accuracy: 1.0000
Epoch 1787/2000
3/3 [==============================] - 0s 76ms/step - loss: 2.8382e-04 - categorical_accuracy: 1.0000
Epoch 1788/2000
3/3 [==============================] - 0s 77ms/step - lo

3/3 [==============================] - 0s 81ms/step - loss: 2.4937e-04 - categorical_accuracy: 1.0000
Epoch 1850/2000
3/3 [==============================] - 0s 80ms/step - loss: 2.4710e-04 - categorical_accuracy: 1.0000
Epoch 1851/2000
3/3 [==============================] - 0s 82ms/step - loss: 2.4666e-04 - categorical_accuracy: 1.0000
Epoch 1852/2000
3/3 [==============================] - 0s 84ms/step - loss: 2.4622e-04 - categorical_accuracy: 1.0000
Epoch 1853/2000
3/3 [==============================] - 0s 83ms/step - loss: 2.4803e-04 - categorical_accuracy: 1.0000
Epoch 1854/2000
3/3 [==============================] - 0s 80ms/step - loss: 2.4979e-04 - categorical_accuracy: 1.0000
Epoch 1855/2000
3/3 [==============================] - 0s 79ms/step - loss: 2.4355e-04 - categorical_accuracy: 1.0000
Epoch 1856/2000
3/3 [==============================] - 0s 78ms/step - loss: 2.4375e-04 - categorical_accuracy: 1.0000
Epoch 1857/2000
3/3 [==============================] - 0s 81ms/step - lo

3/3 [==============================] - 0s 87ms/step - loss: 2.1714e-04 - categorical_accuracy: 1.0000
Epoch 1919/2000
3/3 [==============================] - 0s 88ms/step - loss: 2.1333e-04 - categorical_accuracy: 1.0000
Epoch 1920/2000
3/3 [==============================] - 0s 120ms/step - loss: 2.1393e-04 - categorical_accuracy: 1.0000
Epoch 1921/2000
3/3 [==============================] - 0s 86ms/step - loss: 2.1327e-04 - categorical_accuracy: 1.0000
Epoch 1922/2000
3/3 [==============================] - 0s 83ms/step - loss: 2.1278e-04 - categorical_accuracy: 1.0000
Epoch 1923/2000
3/3 [==============================] - 0s 78ms/step - loss: 2.1254e-04 - categorical_accuracy: 1.0000
Epoch 1924/2000
3/3 [==============================] - 0s 77ms/step - loss: 2.1467e-04 - categorical_accuracy: 1.0000
Epoch 1925/2000
3/3 [==============================] - 0s 77ms/step - loss: 2.1335e-04 - categorical_accuracy: 1.0000
Epoch 1926/2000
3/3 [==============================] - 0s 77ms/step - l

3/3 [==============================] - 0s 91ms/step - loss: 1.8536e-04 - categorical_accuracy: 1.0000
Epoch 1988/2000
3/3 [==============================] - 0s 82ms/step - loss: 1.8635e-04 - categorical_accuracy: 1.0000
Epoch 1989/2000
3/3 [==============================] - 0s 77ms/step - loss: 1.8458e-04 - categorical_accuracy: 1.0000
Epoch 1990/2000
3/3 [==============================] - 0s 78ms/step - loss: 1.8513e-04 - categorical_accuracy: 1.0000
Epoch 1991/2000
3/3 [==============================] - 0s 82ms/step - loss: 1.8526e-04 - categorical_accuracy: 1.0000
Epoch 1992/2000
3/3 [==============================] - 0s 79ms/step - loss: 1.8422e-04 - categorical_accuracy: 1.0000
Epoch 1993/2000
3/3 [==============================] - 0s 76ms/step - loss: 1.8375e-04 - categorical_accuracy: 1.0000
Epoch 1994/2000
3/3 [==============================] - 0s 76ms/step - loss: 1.8421e-04 - categorical_accuracy: 1.0000
Epoch 1995/2000
3/3 [==============================] - 0s 77ms/step - lo

In [81]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 50, 64)            442112    
                                                                 
 lstm_10 (LSTM)              (None, 50, 128)           98816     
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 2)                 66        
                                                                 
Total params: 596,642
Trainable params: 596,642
Non-tr

PREDICCIONES

In [82]:
res = model.predict(X_test)

In [83]:
actions[np.argmax(res[4])]

'colegio'

In [84]:
actions[np.argmax(y_test[4])]

'colegio'

9. Save Weights

In [85]:
#GRABAR MODELO CON EL NOMBRE ACTION.H5
model.save('action.h5')

In [86]:
#CUIDADO, BORRA EL ANTERIOR MODELO 
#del model

In [87]:
#CARGAR EL MODELO NUEVO
model.load_weights('action.h5')

EVALUACION ( Confusion Matrix y Accuracy)

In [88]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [89]:
yhat = model.predict(X_test)

In [90]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [91]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[0, 0],
        [1, 4]],

       [[4, 1],
        [0, 0]]], dtype=int64)

In [92]:
accuracy_score(ytrue, yhat)

0.8

TESTEAR EN TIEMPO REAL

In [93]:
from scipy import stats

In [94]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame


In [152]:
# plt.figure(figsize=(18,18))
# plt.imshow(prob_viz(res, actions, image, colors))

In [96]:
# 1. NUEVA DETECCION DE VARIABLES
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# inicializar holistic
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Lectura de camara
        ret, frame = cap.read()

        # Hacer detecciones
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Dibujar landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-50:]
        
        if len(sequence) == 50:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # mostrar en pantalla
        cv2.imshow('OpenCV Feed', image)

        # parar camara
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

abajo
abajo
abajo
abajo
colegio
abajo
colegio
abajo
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
colegio
abaj